version 1 is for public all other version are experiment

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load/

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
train_targets_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')

In [ ]:
train_targets_scored.head()

In [ ]:
train_features.head()

In [ ]:
test_features.head()

In [ ]:
train_targets_scored.sum()[1 : ].sort_values()

In [ ]:
#targets = ['atp-sensitive_potassium_channel_antagonist', 'erbb2_inhibitor']
#train_targets_scored.loc[:, targets] = 0

In [ ]:
targets = ['atp-sensitive_potassium_channel_antagonist', 'erbb2_inhibitor']
train_targets_scored.loc[:, targets] = 0.000012

In [ ]:
X = train_features.iloc[: ,2:].values
y = train_targets_scored.iloc[:,1:].values
X_final = test_features.iloc[: ,2:].values


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
X[:,1] = le.fit_transform(X[:,1])
X_final[:,1] = le.fit_transform(X_final[:,1])

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
X_final = sc.transform(X_final)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mmsc = MinMaxScaler() 
X = sc.fit_transform(X) 
X_final = sc.transform(X_final) 

Feature Scalling (0,1)

In [ ]:
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer()
X[:,2:] = qt.fit_transform(X[:,2:])
X_final[:,2:] = qt.transform(X_final[:,2:])

In [ ]:
from sklearn.preprocessing import RobustScaler
rs = RobustScaler()
X[:,2:] = rs.fit_transform(X[:,2:])
X_final[:,2:] = rs.transform(X_final[:,2:])


In [ ]:
print(X)
print(X.shape)

In [ ]:
print(X_final)
print(X_final.shape)

In [ ]:
from sklearn.model_selection import train_test_split
#X_1, X_test, y_1, y_test = train_test_split(X, y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

MODEL TENSOR FLOW NEURAL NETWORK

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation,Dense,Dropout,BatchNormalization,Input
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
import tensorflow_addons as tfa

In [ ]:
model = Sequential([
    Input(X_train.shape[1]),
    BatchNormalization(),
    Dropout(0.9),
    tfa.layers.WeightNormalization(Dense(2048, activation="relu")),
    BatchNormalization(),
    Dropout(0.4),
    tfa.layers.WeightNormalization(Dense(1048, activation="relu")),
    BatchNormalization(),
    Dropout(0.2),
    tfa.layers.WeightNormalization(Dense(1048, activation="relu")),
    BatchNormalization(),
    Dropout(0.2),
    tfa.layers.WeightNormalization(Dense(1048, activation="relu")),
    BatchNormalization(),
    Dropout(0.2),
    tfa.layers.WeightNormalization(Dense(y_train.shape[1], activation="sigmoid"))
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
def my_func(arg):
  arg = tf.convert_to_tensor(arg, dtype=tf.float32)
  return arg

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
history = model.fit(
    my_func(X_train), my_func(y_train), verbose=2, epochs=50,
    validation_data=(my_func(X_val), my_func(y_val)),batch_size=32,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.3, 
            patience=3,
            epsilon = 1e-4, 
            mode = 'min',
            verbose=1
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0,
            patience=10,
            mode='auto',
            verbose=1,
            baseline=None,
            restore_best_weights=True
        )
    ]
)

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(0,len(loss_train))
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
epochs = range(0, len(loss_train)) 
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(my_func(X_final))
y_pred = np.clip(y_pred,0.001,0.999) 
columns = list(sample_submission.columns)
columns.remove('sig_id')

for i in range(len(columns)):
    sample_submission[columns[i]] = y_pred[:, i]

sample_submission.to_csv('submission.csv', index=False)